In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hts-data/htsdata_01.csv
/kaggle/input/hts-data/General Notes.pdf
/kaggle/input/hts-data/htsdata_02.csv
/kaggle/input/hts-data/htsdata_04.csv
/kaggle/input/hts-data/htsdata_05.csv
/kaggle/input/hts-data/htsdata_03.csv


In [2]:
# 1. Downgrade numpy & install a SciPy version that supports Python 3.11 and meets gensim’s <1.14 constraint
!pip install --upgrade --force-reinstall --no-deps \
    numpy==1.23.5 \
    scipy==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [3]:
# 2. Install only the libraries you actually need, without pulling their own numpy/scipy versions
!pip install --quiet --no-deps \
  langchain langchain_community \
  duckdb pandas \
  sentence-transformers \
  transformers bitsandbytes \
  faiss-cpu \
  huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.6 MB/s eta 0:00:00:00:0100:01


In [4]:
import duckdb, re, os
import pandas as pd
from huggingface_hub import notebook_login
notebook_login()  # paste your HF token

In [8]:
from langchain.chains import ConversationalRetrievalChain

# LangChain & embeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.tools import BaseTool, Tool

# LLM wrapper
from langchain_community.llms import HuggingFacePipeline
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline as hf_pipeline
)

In [9]:
import torch, gc

# 1. Delete any big tensors or models you don’t need
#    If you stored your model in `model`, uncomment the next line:
# del model

# 2. Empty PyTorch’s CUDA cache
torch.cuda.empty_cache()

# 3. Run Python’s garbage collector
gc.collect()

# 4. (Optional) Print current GPU usage
print("Allocated:", torch.cuda.memory_allocated() / 1e9, "GB")
print("Cached:   ", torch.cuda.memory_reserved()  / 1e9, "GB")


Allocated: 0.0 GB
Cached:    0.0 GB


In [10]:
# Load Llama-2-13b-chat-hf in 4-bit
bnb = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16"
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-13b-chat-hf", quantization_config=bnb, device_map="auto",offload_folder="offload",offload_state_dict=True
)
pipe = hf_pipeline(
    "text-generation", model=model, tokenizer=tokenizer,
    max_new_tokens=256, do_sample=False, repetition_penalty=1.1,eos_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_35/2402301262.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [11]:
# Cell 4: Load all 5 CSVs into DuckDB
con = duckdb.connect(database=":memory:")
con.execute("""
CREATE TABLE hts_section1 AS
SELECT * FROM read_csv_auto('/kaggle/input/hts-data/htsdata_*.csv');
""")
print("Rows loaded:", con.execute("SELECT COUNT(*) FROM hts_section1").fetchone()[0])


Rows loaded: 1669


In [12]:
# Cell 5: Ingest & chunk General Notes PDF, build FAISS
loader = PyPDFLoader("/kaggle/input/hts-data/General Notes.pdf")
docs = loader.load()
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, emb)
print("Embedding count:", vector_store.index.ntotal)


/tmp/ipykernel_35/1226683765.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding count: 878


In [13]:
# Cell 6: Manual RAG-QA tool (avoids bad imports)
def rag_func(query: str) -> str:
    # retrieve top-5 chunks
    docs = vector_store.similarity_search(query, k=5)
    context = "\n---\n".join([d.page_content for d in docs])
    prompt = (
        "Use these HTS General Notes excerpts to answer the question below.\n\n"
        f"{context}\n\nQ: {query}\nA:"
    )
    return llm(prompt)

rag_tool = Tool(
    name="HTSQA",
    func=rag_func,
    description="Answer policy/agreement questions using HTS General Notes."
)


In [14]:
# Cell 7: TariffCalculator tool
def parse_duty_advanced(duty_str, cif, wt=None, qty=None):
    if not duty_str: return 0.0
    s = duty_str.lower()
    if "free" in s: return 0.0
    m = re.search(r"([0-9.]+)%", s)
    if m: return float(m.group(1)) / 100
    m = re.search(r"([0-9.]+)¢/kg", s)
    if m and wt is not None: return (float(m.group(1))*wt/100) / cif
    m = re.search(r"\$([0-9.]+)/unit", s)
    if m and qty is not None: return (float(m.group(1))*qty) / cif
    return 0.0

class TariffCalculator(BaseTool):
    name: str = "TariffCalculator"
    description: str = "Compute duties for an HTS code given cost, freight, insurance, qty, weight"
    con: duckdb.DuckDBPyConnection

    def _run(self, query: str) -> str:
        # parse inputs
        cm = re.search(r"hts code ([\d\.]+)", query, re.I)
        cmc = re.search(r"cost \$?([\d\.]+)", query, re.I)
        fm = re.search(r"freight \$?([\d\.]+)", query, re.I)
        im = re.search(r"insurance \$?([\d\.]+)", query, re.I)
        qm = re.search(r"(\d+)\s*units?", query, re.I)
        wm = re.search(r"([\d\.]+)\s*kg", query, re.I)
        if not cm or not cmc:
            return "❗️ Could not parse HTS code or cost."
        code = cm.group(1)
        cost = float(cmc.group(1))
        freight = float(fm.group(1)) if fm else 0.0
        insurance = float(im.group(1)) if im else 0.0
        qty = int(qm.group(1)) if qm else None
        wt = float(wm.group(1)) if wm else None
        cif = cost + freight + insurance

        row = self.con.execute(f"""
            SELECT "General Rate of Duty", "Special Rate of Duty", "Column 2 Rate of Duty"
            FROM hts_section1
            WHERE "HTS Number" LIKE '{code}%'
            LIMIT 1
        """).fetchone()
        if not row:
            return f"❗️ No data for HTS code {code}."

        rates = list(row)
        fracs = [parse_duty_advanced(r, cif, wt, qty) for r in rates]
        amts  = [f * cif for f in fracs]
        total = sum(amts)
        landed = cif + total

        out = [
            f"**HTS Code:** {code}",
            f"**CIF:** ${cif:,.2f}",
            f"- General: {rates[0]} → {fracs[0]*100:.2f}% → ${amts[0]:.2f}",
            f"- Special: {rates[1]} → {fracs[1]*100:.2f}% → ${amts[1]:.2f}",
            f"- Column 2: {rates[2]} → {fracs[2]*100:.2f}% → ${amts[2]:.2f}",
            f"**Total Duties:** ${total:,.2f}",
            f"**Landed Cost:** ${landed:,.2f}",
        ]
        return "\n".join(out)

    async def _arun(self, query: str) -> str:
        return self._run(query)

tariff_calculator = TariffCalculator(con=con)
tariff_tool = Tool(
    name=tariff_calculator.name,
    func=tariff_calculator._run,
    description=tariff_calculator.description
)


In [25]:
# ─────────────────────────────────────────────────────────────────────────────
# 1) One-shot demo examples
# ─────────────────────────────────────────────────────────────────────────────
rag_demo = """
User: What is the United States-Israel Free Trade Agreement?
Thought: The user is asking about a trade agreement, so I should call the HTSQA tool.
Action: HTSQA
Action Input: What is the United States-Israel Free Trade Agreement?

Observation: The United States–Israel Free Trade Agreement (FTA), effective on September 1, 1985, eliminates tariffs on most goods traded between the two countries while preserving certain safeguards on agricultural products.
Thought: I have the relevant details from the General Notes.
Final Answer: The US–Israel FTA, in force since 1985, removes duties on the vast majority of industrial and many agricultural goods; you can refer to General Note 3(f) for specifics.
""".strip()

tariff_demo = """
User: HTS code 0101.30.00.00, cost $10,000, freight $500, insurance $100, 5 units, 500 kg.
Thought: This is a product-detail request, so I should call the TariffCalculator.
Action: TariffCalculator
Action Input: HTS code 0101.30.00.00, cost $10,000, freight $500, insurance $100, 5 units, 500 kg.

Observation: **HTS Code:** 0101.30.00.00  
**CIF Value:** $10,600.00  
- General Rate: 2% → Duty = $212.00  
- Special Rate: free → Duty = $0.00  
- Column 2 Rate: free → Duty = $0.00  

**Total Duties:** $212.00  
**Landed Cost (CIF + Duties):** $10,812.00  

Thought: I now have the full duty breakdown and landed cost.  
Final Answer: Your total duties are $212.00, making the landed cost $10,812.00.
""".strip()


# ─────────────────────────────────────────────────────────────────────────────
# 2) MRKL format instructions
# ─────────────────────────────────────────────────────────────────────────────
format_instructions = """
Use this format exactly:

Thought: <which tool you will call>
Action: <tool name — HTSQA or TariffCalculator>
Action Input: <the input to that tool>
Observation: <the tool’s raw output>
…repeat Thought/Action/Action Input/Observation as needed…
Final Answer: <the answer you present to the user>
""".strip()


# ─────────────────────────────────────────────────────────────────────────────
# 3) Build the PromptTemplate
# ─────────────────────────────────────────────────────────────────────────────
from langchain.prompts import PromptTemplate

tool_descriptions = "\n".join(f"{t.name}: {t.description}" for t in (rag_tool, tariff_tool))

template = PromptTemplate(
    input_variables=["input"],
    template=(
        "{rag_demo}\n\n"
        "{tariff_demo}\n\n"
        "Tools:\n{tool_descriptions}\n\n"
        "{format_instructions}\n\n"
        "User: {input}\n"
        "Thought:"
    )
)

prompt = template.partial(
    rag_demo=rag_demo,
    tariff_demo=tariff_demo,
    tool_descriptions=tool_descriptions,
    format_instructions=format_instructions
)


# ─────────────────────────────────────────────────────────────────────────────
# 4) MRKLChain Setup & Execution
# ─────────────────────────────────────────────────────────────────────────────
from langchain.agents.mrkl.base import MRKLChain, ChainConfig

configs = [
    ChainConfig(action_name=rag_tool.name,        action=rag_tool.func,       action_description=rag_tool.description),
    ChainConfig(action_name=tariff_tool.name,    action=tariff_tool.func,    action_description=tariff_tool.description),
]

agent_executor = MRKLChain.from_chains(
    llm=llm,
    chains=configs,
    prompt=prompt,
    handle_parsing_errors=True,
    verbose=True,
    max_iterations=2
)

# Test your agent:

#RAG Agent Test:

#print(agent_executor.run("What is the United States-Israel Free Trade Agreement?"))
#print(agent_executor.run("Can a product that exceeds its tariff-rate quota still qualify for duty-free entry under GSP or any FTA? Why or why not?"))
#print(agent_executor.run("How is classification determined for an imported item that will be used as a part in manufacturing but isn’t itself a finished part?"))

#Tariff Agent Test:

#print(agent_executor.run(
 #   "HTS code 0101.30.00.00, cost $10,000, freight $500, insurance $100, 5 units, 500 kg."
#))
#print(agent_executor.run("What’s the HTS Number for donkeys?"))
print(agent_executor.run("What are the applicable duty rates for female cattle? "))




> Entering new MRKLChain chain...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

HTSQA(query: str) -> str - Answer policy/agreement questions using HTS General Notes.
TariffCalculator(query: str) -> str - Compute duties for an HTS code given cost, freight, insurance, qty, weight

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [HTSQA, TariffCalculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What are the applicable duty rates for female cattle? 
Thought: Hmmm, let me check the Harmonized Tariff Schedule...
Action: HTSQA("female cattle")
Action Input: female cattle
Obse

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

HTSQA(query: str) -> str - Answer policy/agreement questions using HTS General Notes.
TariffCalculator(query: str) -> str - Compute duties for an HTS code given cost, freight, insurance, qty, weight

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [HTSQA, TariffCalculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What are the applicable duty rates for female cattle? 
Thought:Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you 